In [59]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os

from selenium import webdriver

In [46]:
years = list(range(1991, 2021))
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

# Scrape MVPs

In [47]:
dfs = []

# Already fetched
# for year in years:
#   url = url_start.format(year)
#   data = requests.get(url)
#   file_name = "./mvps/{}.html".format(year)


#   if os.path.exists(file_name):
#     os.remove(file_name)
#   with open(file_name, "w+") as f:
#     f.write(data.text)
  

In [48]:
for year in years:
  with open("./mvps/{}.html".format(year)) as f:
    page = f.read()
  
  soup = bs(page, "html.parser")
  soup.find("tr", class_="over_header").decompose()
  mvp_table = soup.find_all(id="mvp")
  mvp = pd.read_html(str(mvp_table))[0]
  mvp["Year"] = year

  print(year)

  dfs.append(mvp)
  mvps = pd.concat(dfs)


/tmp/ipykernel_190103/2205256390.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


1991


/tmp/ipykernel_190103/2205256390.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


1992


/tmp/ipykernel_190103/2205256390.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


...

/tmp/ipykernel_190103/2205256390.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


2018


/tmp/ipykernel_190103/2205256390.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


2019
2020


/tmp/ipykernel_190103/2205256390.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp = pd.read_html(str(mvp_table))[0]


In [49]:
if os.path.exists("mvps.csv"):
  os.remove("mvps.csv")
mvps.to_csv("mvps.csv")

# Scrape player stats

In [ ]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

In [60]:
driver = webdriver.Chrome()

The chromedriver version (122.0.6187.0) detected in PATH at /home/vscode/.local/lib/python3.10/site-packages/chromedriver_binary/chromedriver might not be compatible with the detected chrome version (120.0.6099.109); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x562fbf877453 <unknown>
#1 0x562fbf55ebd6 <unknown>
#2 0x562fbf59314c <unknown>
#3 0x562fbf58edfd <unknown>
#4 0x562fbf5d878c <unknown>
#5 0x562fbf5cbdd3 <unknown>
#6 0x562fbf59ca33 <unknown>
#7 0x562fbf59d44e <unknown>
#8 0x562fbf839ccb <unknown>
#9 0x562fbf83dc7a <unknown>
#10 0x562fbf826735 <unknown>
#11 0x562fbf83e8ef <unknown>
#12 0x562fbf80a73f <unknown>
#13 0x562fbf864948 <unknown>
#14 0x562fbf864b13 <unknown>
#15 0x562fbf8765f4 <unknown>
#16 0x7f775b98d609 start_thread
